In [20]:
import csv
import json
import pprint
import gzip
from collections import defaultdict
import string

pathAmazon = "C:/Users/Ian/Documents/PythonDataProducts4PredictiveAnalytics/BasicDataProcessing&Visualization/Final_Course1/datasets/amazon_reviews_us_Gift_Card_v1_00.tsv.gz"

In [4]:
fileAmazon = gzip.open(pathAmazon, 'rt', encoding="utf8")
dataAmazon = csv.reader(fileAmazon, delimiter='\t') 
header = next(dataAmazon)

In [5]:
datasetAmazon = []
for line in dataAmazon:
    # Construimos el diccionario dict(zip(clave, valor))
    d = dict(zip(header, line))
    for field in ['helpful_votes', 'star_rating', 'total_votes']:
        d[field] = int(d[field])
    for field in ['verified_purchase', 'vine']:
        if(d[field] == 'Y'):
            d[field] = True
        else:
            d[field] = False
    datasetAmazon.append(d)

La estructura de nuestros datos se ve así:

In [6]:
print("Total de datos: ", len(datasetAmazon))
pprint.pprint(datasetAmazon[0])

Total de datos:  148310
{'customer_id': '24371595',
 'helpful_votes': 0,
 'marketplace': 'US',
 'product_category': 'Gift Card',
 'product_id': 'B004LLIL5A',
 'product_parent': '346014806',
 'product_title': 'Amazon eGift Card - Celebrate',
 'review_body': 'Great birthday gift for a young adult.',
 'review_date': '2015-08-31',
 'review_headline': 'Five Stars',
 'review_id': 'R27ZP1F1CD0C3Y',
 'star_rating': 5,
 'total_votes': 0,
 'verified_purchase': True,
 'vine': False}


# Filtrado de datos  
Se trabajara los datos en formato de diccionario para una mejor manipulación de los datos 

### Filtrado por fecha
Para esto, vamos a convertir el campo review_date en integer, tomar solo el año y crear un nuevo campo llamado _yearInt_ para filtrar los datos más recientes.  
**Para esto, devemos de realizar un preprocesamiento ya que algunos documentos no tienen ningún valor en ese campo**

In [7]:
cleanDSA = [d for d in datasetAmazon if 'review_date' in d]
print(len(cleanDSA))

148309


In [10]:
for d in cleanDSA:
    d['yearInt'] = int(d['review_date'][:4])

Limpio nuestro conjunto de datos, vamos a realizar el filtrado de opiniones que realizadas a partir del 2010

In [12]:
cleanDSA = [d for d in cleanDSA if d['yearInt'] > 2009]
print(len(cleanDSA))

148095


### Filtramos las opiniones con mayor relevancia.  
Para esto consideramos dos parámetros:  
* Tiene menos de tres votos de utilidad. Si tienen pocos votos, la escencia de la relevancia no es muy acertada
* Su relación de utilidad es mayor que 0.5. Su radio de ayuda es mayor a 0.5

In [13]:
cleanDSA = [d for d in cleanDSA if d['total_votes'] < 3  or d['helpful_votes']/d['total_votes'] >= 0.5]
print(len(cleanDSA))

147801


### Filtramos por actividad de usuario.  
Para esto, realizamos un conteo de las apariciones de los usuarios que más comentarios tienen

In [14]:
nReviewsperUser = defaultdict(int)
for d in cleanDSA:
    nReviewsperUser[d['customer_id']] += 1

### Filtramos a los usuarios que tengan como mínimo dos reseñas  
Para esto, realizamos el conteo de número de apariciones en el conjunto de datos

In [17]:
cleanDSA= [d for d in cleanDSA if nReviewsperUser[d['customer_id']] >= 2]
print(len(cleanDSA))

11172


### Filtramos por longitud de la reseña
Realizamos un filtro de reseñas que son muy cortas (10 palabras o menos) y para esto, separamos las palabras y realizamos un conteo

In [18]:
cleanDSA = [d for d in cleanDSA if len(d['review_body'].split()) >= 10]
print(len(cleanDSA))

7033


In [21]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'